In [29]:
import pandas as pd
import re

# Charger le fichier
with open("tesis.txt", "r") as file:
    content = file.read()

# Séparer les paragraphes par '\n\n'
paragraphs = content.split('\n\n')

# Initialiser une liste vide
data = []

# Boucle à travers chaque paragraphe pour faire le tri
i = 0
while i < len(paragraphs):
    para = paragraphs[i]
    
    # Supprimer les paragraphes vides
    if para.strip() == '':
        i += 1
        continue
    # Si un paragraphe commence par “, il doit être contenu dans une seule et même ligne du dataset jusqu'à la fermeture des guillemets par ”
    elif para.startswith('“'):
        while '”' not in para:
            i += 1
            if i < len(paragraphs):
                # Vérifier si le paragraphe suivant ne commence pas par un guillemet ouvrant
                if not paragraphs[i].startswith('“'):
                    para += ' ' + paragraphs[i]
                else:
                    break
            else:
                break
    # Ignorer les paragraphes commençant par "• ", "❖ " ou "o "
    elif para.startswith("•	") or para.startswith("❖	") or para.startswith("o	"):
        i += 1
        continue

    # Ajouter le paragraphe nettoyé à la liste
    data.append([para])
    i += 1

# Convertir la liste en DataFrame
df = pd.DataFrame(data, columns=['Paragraph'])

# Boucle à travers chaque paragraphe pour supprimer une ou deux lettres suivies d'une tabulation
for index, row in df.iterrows():
    cleaned_para = re.sub(r'^.{1,2}\t', '', row['Paragraph'])
    df.at[index, 'Paragraph'] = cleaned_para

# Afficher le DataFrame
print(df)

                                             Paragraph
0    El interés por redactar el presente proyecto d...
1    Ahora bien, de conformidad con la premisa ante...
2    No obstante, previo a desarrollarla, resulta r...
3    Dicho de otro modo, en virtud de que el ser hu...
4    En este orden de ideas, debemos tener en mente...
..                                                 ...
388  “Artículo 153-C. El adiestramiento tendrá por ...
389  Adicionalmente, con relación a la velocidad co...
390  “Artículo 7º.- En toda empresa o establecimien...
391  Finalmente y a modo de reflexión para el lecto...
392  “Algunas veces, las estrategias son más import...

[393 rows x 1 columns]


In [30]:
df.to_csv('tesis.csv')

In [32]:
import os
import openai
import time
from ast import literal_eval

openai.organization = os.environ['OPENAI_ORG_KEY']
openai.api_key = os.environ['OPENAI_API_KEY']

In [57]:
def content_eval(content):
    role = """Soy GPT-derecho-si-no. Mi función es responder con una sola palabra a la pregunta "¿Tiene este párrafo algo que ver con una tesis en derecho?". Si tiene que ver con el derecho, respondo que sí; si tiene que ver con otra cosa que el derecho, respondo que no.""" 
    while True:
        try:
            completions = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                { "role": "assistant", "content": role
                },
                { "role": "user", "content": content}
                ],
                max_tokens = 2,
            )
            return completions['choices'][0]['message']['content']
        except openai.error.RateLimitError:  # handle the specific RateLimitError
            print('API overload, waiting for 10 seconds...')
            time.sleep(10)
        except Exception as e:
            # Handle any other exceptions that might occur
            print("Error:", str(e))
            time.sleep(10)

In [ ]:
from tqdm import tqdm

# Appliquer la fonction à chaque ligne de votre DataFrame si la cellule de la nouvelle colonne est NaN
for i in tqdm(range(df.shape[0])):
    df['Content_Eval'] = df['Paragraph'].apply(content_eval)


In [60]:
content_eval(df['Paragraph'][30])

'Sí'